# ResNet50 pre-modeing
- 사용할 데이터 : Cifar-100 dataset

## Cifar-100 데이터 준비
- Cifar-100 dataset은 32 x 32 x 3크기의 50,000개 training-set이 있다.
- Cifar-100 dataset은 label은 100개의 class로 나누어져 있다.
- Cifar-100 dataset 훈련,검증,테스트 집합으로 나눈다.
- 기존 데이터에서 일부만 선택해서 사용
train : 192000, validation : 4800, test : 6000개
- labels의 경우 one-hot Encoding 방식을 적용한다.
- ResNet50의 Input size를 맞추기 위해 원본 이미지 크기인 229 x 229 x 3을 224 x 224 x 3으로 resize



In [ ]:
#공통 임포트 모듈
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [ ]:
#데이터를 훈련 집합과 테스트 집합으로 나누기
from sklearn.model_selection import train_test_split
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar100.load_data()

x_train,x_valid,y_train,y_valid = train_test_split(x_train, y_train, test_size=0.2,random_state=123)

# 기존 데이터에서 일부만 선택해서 사용 
# train : 192000, validation : 4800, test : 6000개
x_train = x_train[:19200]
y_train = y_train[:19200]

x_valid = x_valid[:4800]
y_valid = y_valid[:4800]

x_test = x_test[:6000]
y_test = y_test[:6000]

# X_train,X_valid,X_test 모두 224 x 224 x 3 크기로 resize

#resize (32,32) - > (224,224)

# x_train의 경우
data = []
for idx in range(len(x_train)):
  # \r은 해당 줄의 처음으로 이동,end=''은 줄을 개행하지 않고 출력
  print('\r start ',idx+1,'/',len(x_train),end='')
  img = x_train[idx]
  img = cv2.resize(img,(224,224))
  data.append(img)
print()#end

x_train = np.array(data)
print(x_train.shape)

# x_valid의 경우
data = []
for idx in range(len(x_valid)):
  # \r은 해당 줄의 처음으로 이동,end=''은 줄을 개행하지 않고 출력
  print('\r start ',idx+1,'/',len(x_valid),end='')
  img = x_valid[idx]
  img = cv2.resize(img,(224,224))
  data.append(img)
print()#end

x_valid = np.array(data)
print(x_valid.shape)

# x_test의 경우
data = []
for idx in range(len(x_test)):
  # \r은 해당 줄의 처음으로 이동,end=''은 줄을 개행하지 않고 출력
  print('\r start ',idx+1,'/',len(x_test),end='')
  img = x_test[idx]
  img = cv2.resize(img,(224,224))
  data.append(img)
print()#end

x_test = np.array(data)
print(x_test.shape)


# label의 경우 one-hot Encoding 
y_train = tf.squeeze(tf.one_hot(y_train,100),axis=1)
y_valid = tf.squeeze(tf.one_hot(y_valid,100),axis=1)
y_test = tf.squeeze(tf.one_hot(y_test,100),axis=1)

print('train data')
print(x_train.shape)
print(y_train.shape)

print('validation data')
print(x_valid.shape)
print(y_valid.shape)

print('test data')
print(x_test.shape)
print(y_test.shape)

169009152/169001437 [==============================] - 4s 0us/step
 start  19200 / 19200
(19200, 224, 224, 3)
 start  4800 / 4800
(4800, 224, 224, 3)
 start  6000 / 6000
(6000, 224, 224, 3)
train data
(19200, 224, 224, 3)
(19200, 100)
validation data
(4800, 224, 224, 3)
(4800, 100)
test data
(6000, 224, 224, 3)
(6000, 100)


## ResNet50 모델생성

- 1.ResNet50을 ImageNet 데이터셋으로 학습시킨 weight와 bias를 사용하여 pre-modeling을 한다.

- 2.ResNet50의 모델을 1에서 구한 ResNet50을 input으로하고 1에서 구한 ResNet50의 average pooling층을 output으로 하는 모델을 새롭게 구성

- 3.Cifar-100 dataset은 class가 100개이므로 FC(Fully-connected)100을 생성
- 4.2에서 구한 모델을 input으로하고 3에서 구한 FC 100를 output으로 하는 새로운 모델을 생성
- 5.Fine-tunning의 방법으로 마지막 FC Layer만 재학습시킨다.

In [ ]:
#ResNet50
#Fine-Tuning 
base_model = tf.keras.applications.ResNet50(weights='imagenet',input_shape=(224,224,3))

base_model = tf.keras.models.Model(base_model.inputs,base_model.layers[-2].output)

base_model.trainable = False # 이미 학습이 되어있으므로 추가 학습을 시키지 않는다.

x = base_model.output
pred = tf.keras.layers.Dense(100,activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input,outputs=pred)

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['acc'])

model.summary()

102973440/102967424 [==============================] - 1s 0us/step
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________

In [ ]:
history = model.fit(x=x_train, y=y_train,batch_size=128,epochs=30,validation_data=(x_valid,y_valid))

Epoch 1/30
150/150 [==============================] - 64s 429ms/step - loss: 4.2584 - acc: 0.0849 - val_loss: 3.5591 - val_acc: 0.1950
Epoch 2/30
150/150 [==============================] - 67s 446ms/step - loss: 3.1114 - acc: 0.2849 - val_loss: 2.8273 - val_acc: 0.3435
Epoch 3/30
150/150 [==============================] - 67s 445ms/step - loss: 2.5242 - acc: 0.4110 - val_loss: 2.4328 - val_acc: 0.4277
Epoch 4/30
150/150 [==============================] - 67s 446ms/step - loss: 2.1849 - acc: 0.4798 - val_loss: 2.2004 - val_acc: 0.4681
Epoch 5/30
150/150 [==============================] - 67s 446ms/step - loss: 1.9650 - acc: 0.5234 - val_loss: 2.0452 - val_acc: 0.4956
Epoch 6/30
150/150 [==============================] - 67s 446ms/step - loss: 1.8092 - acc: 0.5526 - val_loss: 1.9361 - val_acc: 0.5115
Epoch 7/30
150/150 [==============================] - 67s 447ms/step - loss: 1.6901 - acc: 0.5776 - val_loss: 1.8524 - val_acc: 0.5263
Epoch 8/30
150/150 [==============================] - 6

In [ ]:
results = model.evaluate(x_test,y_test,batch_size=128)

print('test accuracy')
print(results[1])

47/47 [==============================] - 17s 363ms/step - loss: 1.3951 - acc: 0.6138
test accuracy
0.6138333082199097
